<a href="https://colab.research.google.com/github/YINZihua/yinzihua_PRIM_SSD_PyTorch/blob/main/PRIM3A_Image_Vision_SSD_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation for running SSD, cloned from Github project

**TO DO before ALL** launch the training process in order to enable the GPU module:

**Edit > Notebook settings or Runtime>Change runtime type and select GPU as Hardware accelerator**

## Clone from Github the pytorch version SSD project

In [1]:
!git clone https://github.com/amdegroot/ssd.pytorch
%cd ssd.pytorch/

Cloning into 'ssd.pytorch'...
remote: Enumerating objects: 1451, done.
remote: Total 1451 (delta 0), reused 0 (delta 0), pack-reused 1451
Receiving objects: 100% (1451/1451), 104.36 MiB | 39.68 MiB/s, done.
Resolving deltas: 100% (868/868), done.
/content/ssd.pytorch


## Dataset

The VOC challenge database goes down like all the time. We then found an alternative for the same train/test dataset. The usage requires some modification in the scripts to run, such as \\

*  **VOC2012.sh**: <br> (line 25) gdown https://drive.google.com/uc?id=1c8B8cKWlhlx4JK7k-27UumtzXUsfy9wP
*  **VOC2007.sh**: <br> (line 25) gdown https://drive.google.com/uc?id=16IKsZB1KUddfzjIdIPwcyn9kRkdwTPtm
<br>             (line 27) gdown https://drive.google.com/uc?id=1amRirwRVyZ8N_bN4BlVDH7QU7AHXecQ8

We can modify manually, or use the **following modification scripts (if it works)**. 
<br>When running the modification, don't open the download scripts, whereas the modification would be invalidated.



In [2]:
# !pip install gdown 

In [3]:
!sed -i '25c gdown https://drive.google.com/uc?id=1c8B8cKWlhlx4JK7k-27UumtzXUsfy9wP' data/scripts/VOC2012.sh
!sed -i '25c gdown https://drive.google.com/uc?id=16IKsZB1KUddfzjIdIPwcyn9kRkdwTPtm' data/scripts/VOC2007.sh
!sed -i '27c gdown https://drive.google.com/uc?id=1amRirwRVyZ8N_bN4BlVDH7QU7AHXecQ8' data/scripts/VOC2007.sh 

In [4]:
!sh data/scripts/VOC2007.sh
!sh data/scripts/VOC2012.sh

流式输出内容被截断，只能显示最后 5000 行内容。
VOCdevkit/VOC2012/SegmentationClass/2008_001882.png
VOCdevkit/VOC2012/SegmentationClass/2008_001885.png
VOCdevkit/VOC2012/SegmentationClass/2008_001895.png
VOCdevkit/VOC2012/SegmentationClass/2008_001896.png
VOCdevkit/VOC2012/SegmentationClass/2008_001926.png
VOCdevkit/VOC2012/SegmentationClass/2008_001966.png
VOCdevkit/VOC2012/SegmentationClass/2008_001971.png
VOCdevkit/VOC2012/SegmentationClass/2008_001992.png
VOCdevkit/VOC2012/SegmentationClass/2008_001997.png
VOCdevkit/VOC2012/SegmentationClass/2008_002032.png
VOCdevkit/VOC2012/SegmentationClass/2008_002043.png
VOCdevkit/VOC2012/SegmentationClass/2008_002064.png
VOCdevkit/VOC2012/SegmentationClass/2008_002066.png
VOCdevkit/VOC2012/SegmentationClass/2008_002067.png
VOCdevkit/VOC2012/SegmentationClass/2008_002073.png
VOCdevkit/VOC2012/SegmentationClass/2008_002079.png
VOCdevkit/VOC2012/SegmentationClass/2008_002080.png
VOCdevkit/VOC2012/SegmentationClass/2008_002123.png
VOCdevkit/VOC2012/SegmentationClass/2

The two VOC datasets take 102 seconds (1'42'', 65Mbps/sec)

We here will not use the COCO dataset

In [5]:
# !sh data/scripts/COCO2014.sh

The previous scripts has always trouble with path in colab, so we firstly run as it wants, then copy all downloaded dataset to the right path as '(/root)/content/ssd.pytorch/data/..'

In [6]:
!mkdir ~/data/coco/
!cp data/coco_labels.txt ~/data/coco/

The whole COCO download and extraction process have taken 1345 seconds (22'25'', 35.6Mbps/sec).
<br> But I found that the dataset is too large to be used in provided GPU environment on Colab, because **the storage limit is exceeded**!

## Training SSD

### Download the fc-reduced VGG-16 PyTorch base network weights

In [7]:
!mkdir weights
%cd weights
!wget https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth

/content/ssd.pytorch/weights
--2020-11-13 14:44:33--  https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.187.53
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.187.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81938914 (78M) [binary/octet-stream]
Saving to: ‘vgg16_reducedfc.pth’

vgg16_reducedfc.pth 100%[===================>]  78.14M  37.7MB/s    in 2.1s    

2020-11-13 14:44:36 (37.7 MB/s) - ‘vgg16_reducedfc.pth’ saved [81938914/81938914]



In [8]:
%cd ..
!ls

/content/ssd.pytorch
data  doc      layers	README.md  test.py   utils
demo  eval.py  LICENSE	ssd.py	   train.py  weights


### Launch the training process

(Obsolete)**Before run the script on Colab** \\
Due to the different path organization on Colab, we should modify some places in the fonctionnality scripts, such as: \\


*   **coco.py**: COCO_ROOT = osp.join(HOME, '/content/ssd.pytorch/data/coco/')



**Before running the train script, it should modify somewhere in 'train.py' and '../layers/modules/mutibox_loss.py', to avoid the problems dut to the usage of recent version PyTorch on Colab and the usage of COCO dataset which occupies too heavy part of storage, such as :
<br>

**train.py**
*   **(line 26)** parser.add_argument('--dataset', default='VOC', choices=['VOC'],
*   **(line 27)** type=str, help='VOC')
*   **(line 183)** loc_loss += loss_l.item()
*   **(line 184)** conf_loss += loss_c.item()
*   **(line 188)** print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.item()), end=' ')
*   **(line 165)** 
<br>try: 
<br>&emsp;    images, targets = next(batch_iterator)
<br>except StopIteration as e: 
<br>&emsp;    batch_iterator = iter(data_loader) 
<br>&emsp;    images, targets = next(batch_iterator) 

<br>

**layers/modules/multibox_loss.py**

*   **(line 97)** loss_c = loss_c.view(num, -1)
*   **(line 98)** loss_c[pos] = 0 # filter out pos boxes for now
*   **(line 114)** N = num_pos.data.sum().double()





In [9]:
!sed -i "26c parser.add_argument('--dataset', default='VOC', choices=['VOC']," train.py
!sed -i "27c \                    type=str, help='VOC')" train.py
!sed -i '183c \        loc_loss   +=  loss_l.item()' train.py
!sed -i '184c \        conf_loss   +=  loss_c.item()' train.py
!sed -i "188c \            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.item()), end=' ')" train.py
!sed -i "165c \        try:" train.py
!sed -i "165a \            images,targets=next(batch_iterator)" train.py
!sed -i "165a \            batch_iterator=iter(data_loader)" train.py
!sed -i "165a \        except StopIteration as e:" train.py
!sed -i "165a \            images,targets=next(batch_iterator)" train.py

!sed -i "97c \        loss_c = loss_c.view(num, -1)" layers/modules/multibox_loss.py
!sed -i "98c \        loss_c[pos] = 0 # filter out pos boxes for now" layers/modules/multibox_loss.py
!sed -i "114c \        N = num_pos.data.sum().double()" layers/modules/multibox_loss.py

Launch the training !

In [ ]:
!python train.py

/content/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
Loading base network...
Initializing weights...
train.py:218: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(param)
Loading the dataset...
Training SSD on: VOC0712
Using the specified args:
Namespace(basenet='vgg16_reducedfc.pth', batch_size=32, cuda=True, dataset='VOC', dataset_root='/root/data/VOCdevkit/', gamma=0.1, lr=0.001, momentum=0.9, num_workers=4, resume=None, save_folder='weights/', start_iter=0, visdom=False, weight_decay=0.0005)
train.py:173: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be depr

In [ ]:
# !zip ssd300_COCO_25000.pth.zip weights/ssd300_COCO_25000.pth  

  adding: weights/ssd300_COCO_25000.pth (deflated 7%)


In [ ]:
# !ls -lh 

total 94M
drwxr-xr-x 4 root root 4.0K Nov 11 14:51 data
drwxr-xr-x 2 root root 4.0K Nov 11 14:44 demo
drwxr-xr-x 2 root root 4.0K Nov 11 14:44 doc
-rw-r--r-- 1 root root  16K Nov 11 14:44 eval.py
drwxr-xr-x 5 root root 4.0K Nov 11 14:51 layers
-rw-r--r-- 1 root root 1.1K Nov 11 14:44 LICENSE
drwxr-xr-x 2 root root 4.0K Nov 11 14:51 __pycache__
-rw-r--r-- 1 root root 7.2K Nov 11 14:44 README.md
-rw-r--r-- 1 root root  93M Nov 11 23:09 ssd300_COCO_25000.pth.zip
-rw-r--r-- 1 root root 7.5K Nov 11 14:44 ssd.py
-rw-r--r-- 1 root root 3.8K Nov 11 14:44 test.py
-rw-r--r-- 1 root root 9.0K Nov 11 14:51 train.py
drwxr-xr-x 3 root root 4.0K Nov 11 14:51 utils
drwxr-xr-x 2 root root 4.0K Nov 11 22:22 weights


Download the obtained ssd weights file after 25000 rounds of iterations, then I uploaded it on google drive and put it available by using link: https://drive.google.com/file/d/1asBzvW7LOAdyy7MfCw0bPF98vh-zqMdY/view?usp=sharing (id=1asBzvW7LOAdyy7MfCw0bPF98vh-zqMdY)

In [ ]:
# # from google.colab import files
# files.download('ssd300_COCO_25000.pth.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Evaluation SSD

We then need to evaluate the performance of the model with the weights obtained after 25000 iterations, which is accessible from the following download functionality

In [12]:
# https://drive.google.com/file/d/1asBzvW7LOAdyy7MfCw0bPF98vh-zqMdY/view?usp=sharing

from google_drive_downloader import GoogleDriveDownloader as gdd

# downloading original data set 
gdd.download_file_from_google_drive(file_id='1asBzvW7LOAdyy7MfCw0bPF98vh-zqMdY', dest_path='./weights/ssd300_COCO_25000.pth.zip')

# unzip the downloaded file to obtain the .pth weight file
!unzip -qq weights/ssd300_COCO_25000.pth.zip  weights/ssd300_COCO_25000.pth
!rm weights/ssd300_COCO_25000.pth.zip

However, we need still to modify a little bit the evaluation scripts to make it compatible with the colab environment (particularly with the deprecated non-static forward method issues), as following: 

In [70]:
!sed -i "39c \                    default='weights/ssd300_COCO_25000.pth', type=str," eval.py

!sed -i '12c \    ' layers/functions/detection.py
!sed -i '23c \    """' layers/functions/detection.py
!sed -i '24c \    def forward(self, num_classes, bkg_label, top_k, conf_thresh, nms_thresh, loc_data, conf_data, prior_data):' layers/functions/detection.py
!sed -i "33a \        self.variance = cfg['variance']" layers/functions/detection.py
!sed -i "33a \        self.conf_thresh = conf_thresh" layers/functions/detection.py
!sed -i "33a \            raise ValueError('nms_threshold must be non negative.')" layers/functions/detection.py
!sed -i "33a \        if nms_thresh <= 0:" layers/functions/detection.py
!sed -i "33a \        self.nms_thresh = nms_thresh" layers/functions/detection.py
!sed -i "33a \        # Parameters used in nms." layers/functions/detection.py
!sed -i "33a \        self.top_k = top_k" layers/functions/detection.py
!sed -i "33a \        self.background_label = bkg_label" layers/functions/detection.py
!sed -i "33a \        self.num_classes = num_classes" layers/functions/detection.py
!sed -i "23a \    @staticmethod" layers/functions/detection.py

# ssd.py __init__()
!sed -i "47c \            self.softmax = nn.Softmax()" ssd.py
!sed -i "48c \            self.detect = Detect()" ssd.py
# ssd.py forward()
!sed -i "99c \            output = self.detect.apply(21, 0, 200, 0.01, 0.45," ssd.py
!sed -i "101c \                self.softmax(conf.view(-1," ssd.py
!sed -i "102c \                             21)),                # conf preds" ssd.py


Launch the evaluation!

In [71]:
!python eval.py

流式输出内容被截断，只能显示最后 5000 行内容。
im_detect: 76/4952 0.068s
im_detect: 77/4952 0.121s
im_detect: 78/4952 0.145s
im_detect: 79/4952 0.051s
im_detect: 80/4952 0.095s
im_detect: 81/4952 0.119s
im_detect: 82/4952 0.145s
im_detect: 83/4952 0.018s
im_detect: 84/4952 0.048s
im_detect: 85/4952 0.100s
im_detect: 86/4952 0.050s
im_detect: 87/4952 0.046s
im_detect: 88/4952 0.123s
im_detect: 89/4952 0.079s
im_detect: 90/4952 0.117s
im_detect: 91/4952 0.068s
im_detect: 92/4952 0.034s
im_detect: 93/4952 0.102s
im_detect: 94/4952 0.024s
im_detect: 95/4952 0.112s
im_detect: 96/4952 0.143s
im_detect: 97/4952 0.139s
im_detect: 98/4952 0.120s
im_detect: 99/4952 0.077s
im_detect: 100/4952 0.030s
im_detect: 101/4952 0.129s
im_detect: 102/4952 0.131s
im_detect: 103/4952 0.132s
im_detect: 104/4952 0.110s
im_detect: 105/4952 0.182s
im_detect: 106/4952 0.150s
im_detect: 107/4952 0.105s
im_detect: 108/4952 0.028s
im_detect: 109/4952 0.178s
im_detect: 110/4952 0.135s
im_detect: 111/4952 0.142s
im_detect: 112/4952 0.074

We've got a trained model with scored mAP 0.7089

## (obsolete) Compile the project files for caffe

In [ ]:
!cp Makefile.config.example Makefile.config

In [ ]:
!make -j 8 all
!make -j 8 runtest
!make -j 8 pycaffe

PROTOC src/caffe/proto/caffe.proto
CXX src/caffe/net.cpp
CXX src/caffe/data_transformer.cpp
CXX src/caffe/data_reader.cpp
CXX src/caffe/solvers/adagrad_solver.cpp
CXX src/caffe/solvers/adadelta_solver.cpp
CXX src/caffe/syncedmem.cpp
CXX src/caffe/solvers/sgd_solver.cpp
CXX src/caffe/solvers/rmsprop_solver.cpp
src/caffe/net.cpp:8:10: fatal error: hdf5.h: No such file or directory
 #include "hdf5.h"
          ^~~~~~~~
compilation terminated.
Makefile:575: recipe for target '.build_release/src/caffe/net.o' failed
make: *** [.build_release/src/caffe/net.o] Error 1
make: *** Waiting for unfinished jobs....
In file included from src/caffe/syncedmem.cpp:1:0:
./include/caffe/common.hpp:5:10: fatal error: gflags/gflags.h: No such file or directory
 #include <gflags/gflags.h>
          ^~~~~~~~~~~~~~~~~
compilation terminated.
Makefile:575: recipe for target '.build_release/src/caffe/syncedmem.o' failed
make: *** [.build_release/src/caffe/syncedmem.o] Error 1
In file included from src/caffe/data

# (obsolete) Download the training model

In [ ]:
%cd data/
!wget  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

/content/caffe/data
--2020-11-09 10:28:23--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... failed: No route to host.
--2020-11-09 10:28:25--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... failed: No route to host.
--2020-11-09 10:28:29--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... failed: No route to host.


In [ ]:
!ls

cifar10  coco  ilsvrc12  ILSVRC2016  mnist  VOC0712
